In [ ]:
# Code for pulling search words from a google sheet instead of user input
# this is in preperation to set up schedule running of this script to auto update a database.

import gspread
import time
from datetime import date
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import snowflake.connector
from oauth2client.service_account import ServiceAccountCredentials
from gspread.exceptions import WorksheetNotFound


# Define the scope
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']

# Add your Service Account File
creds = ServiceAccountCredentials.from_json_keyfile_name(r"PATH_TO_CREDENTIALS", scope)

# authorize your client
client = gspread.authorize(creds)


def get_search_word_from_sheet():

    # Open the Google Spreadsheet by its url (make sure you have access to it)
    sheet = client.open_by_url('URL_FOR_GOOGLE_SHEET').sheet1

    # Get all the records of the data
    data = sheet.get_all_records()

    # # Get today's day of the week
    # today = datetime.today().strftime('%A')

    # # Checks for day of the week from google sheet
    # for row in data:
    #     # If the day in the 'day_of_week' column matches today's day, return the corresponding search word
    #     if row['day_of_week'] == today:
    #         return row['search_word']

    # Get today's date if searching by Date instead of day of week
    today = date.today()

    # Checks for date instead of day of the week
    for row in data:
        # If the date in the 'date' column matches today's date, return the corresponding search word
        date_from_sheet = datetime.strptime(row['date'], '%m/%d/%Y').date()
        if date_from_sheet == today:
            return row['search_word']

    return None

# Scraping rules for the two websites
def scrape_website(url, word):
    driver = webdriver.Chrome()
    driver.get(url)

    if "livingspaces.com" in url:
        input_field = driver.find_element(By.ID, 'search')
        input_field.send_keys(word)
        form = input_field.find_element(By.XPATH, './ancestor::form')
        form.submit()
    elif "rcwilley.com" in url:
        input_field = driver.find_element(By.ID, 'searchBox')
        input_field.send_keys(word)
        submit_button = driver.find_element(By.ID, 'searchSubmit')
        submit_button.click()

    time.sleep(5)  
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    prices = []

    if "livingspaces.com" in url:
        product_items = soup.find_all('div', class_='product-item-container')
        for item in product_items:
            name_element = item.find('span', class_='name')
            price_element = item.find('span', class_='price')
            rating_element = item.find('div', class_='ratings')
            if name_element and price_element and rating_element:
                name = name_element.text.strip()
                price = price_element.text.strip()

                rating = 0
                rating_text = rating_element['aria-label']
                if rating_text:
                    rating = float(rating_text.split(' out of ')[0])
                rating = int(rating) if isinstance(rating, float) and rating.is_integer() else rating 
                # tried about 40 ways to get int and float to carry over. Snowflake had issues with 'rating' values. Turns out it was my google sheet with empty values. this still has extra checks just incase.

                prices.append({'name': name, 'price': float(price.replace('$', '').replace(',', '')), 'rating': rating})

    elif "rcwilley.com" in url:
        product_items = soup.find_all('div', class_='productContent')
        for item in product_items:
            name_element = item.find('div', class_='productName')
            price_element = item.find('span', class_='price')
            if name_element and price_element:
                name = name_element.text.strip()
                price = price_element.text.strip()
                rating_element = item.find('div', class_='rating')

                rating = 0
                if rating_element:
                    rating_span = rating_element.find('span', class_='sr-only')
                    if rating_span:
                        rating_text = rating_span.text.strip()
                        rating = float(''.join([i for i in rating_text if i.isdigit() or i == '.']))
                rating = int(rating) if isinstance(rating, float) and rating.is_integer() else rating 
                # tried about 40 ways to get int and float to carry over. Snowflake had issues with 'rating' values. Turns out it was my google sheet with empty values. this still has extra checks just incase.
                prices.append({'name': name, 'price': float(price.replace('$', '').replace(',', '')), 'rating': rating})

    driver.quit()

    return prices

# Calculating averages of items scraped
def calculate_average_value(values):
    values_float = [value for value in values if value is not None and value != 0]
    average = sum(values_float) / len(values_float) if values_float else 0
    return round(average, 2) if average is not None else 0

# Get both min and max prices from the item list
def get_min_max(prices):
    if not prices:
        return None, None
    min_price = min(prices, key=lambda x: x['price'])
    max_price = max(prices, key=lambda x: x['price'])
    return min_price, max_price

# Double check the IDs and continue counting
def get_max_id(sheet_name):
    # authorize your client
    client = gspread.authorize(creds)

    # Open the Google Spreadsheet by its url (make sure you have access to it)
    sheet = client.open_by_url('URL_FOR_GOOGLE_SHEET')

    try:
        ws = sheet.worksheet(sheet_name)
        data = ws.get_all_values()  # get all values inside the specified worksheet
        if data:
            # first column has ID, get max value
            ids = [int(row[0]) for row in data[1:] if row[0].isdigit()]  # ignore first row(header)
            if ids:
                return max(ids)
    except WorksheetNotFound:
        pass
    return 0

# Function to add searchID to each search so we can better track our info over time.
def get_search_id():
    search_id = 0
    sheets = ['item_prices', 'compared_prices']

    # Open the Google Spreadsheet by its url
    sheet = client.open_by_url('URL_FOR_GOOGLE_SHEET')

    for sheet_name in sheets:
        try:
            ws = sheet.worksheet(sheet_name)
            data = ws.get_all_values()  # get all values inside the specified worksheet
            if data:
                # first column has search_id, get max value
                search_ids = [int(row[1]) for row in data[1:] if row[1].isdigit()]  # ignore first row(header)
                if search_ids:
                    search_id = max(max(search_ids), search_id)
        except WorksheetNotFound:
            pass
    return search_id + 1


# Final function that prints the comparisons, and adds them to the excel file
def compare_prices(word, item_prices_max_id, compared_prices_max_id):

    website1_url = 'https://www.livingspaces.com/'
    website2_url = 'https://www.rcwilley.com/'

    prices_website1 = scrape_website(website1_url, word)
    prices_website2 = scrape_website(website2_url, word)

    # Calculate average prices
    average_website1 = calculate_average_value([price['price'] for price in prices_website1])
    average_website2 = calculate_average_value([price['price'] for price in prices_website2])

    # Calculate average ratings
    average_rating_website1 = calculate_average_value([float(price['rating']) for price in prices_website1 if price['rating'] is not None])
    average_rating_website2 = calculate_average_value([float(price['rating']) for price in prices_website2 if price['rating'] is not None])


    website1_name = website1_url.replace('https://www.', '').replace('.com/', '').capitalize()
    website2_name = website2_url.replace('https://www.', '').replace('.com/', '').capitalize()

    print(f"Item: {search_word}")
    print()

    print(f"Number of {word}s on {website1_name} front page: {len(prices_website1)}")
    print(f"Average price on {website1_name}: ${average_website1:.2f}")
    min_price, max_price = get_min_max(prices_website1)
    if min_price and max_price:
        print(f"Lowest price on {website1_name}: {min_price['name']} at ${min_price['price']:.2f}")
        print(f"Highest price on {website1_name}: {max_price['name']} at ${max_price['price']:.2f}")
    print(f"Average rating on {website1_name}: {average_rating_website1:.2f}")

    print(f"\nNumber of {word}s on {website2_name} front page: {len(prices_website2)}")
    print(f"Average price on {website2_name}: ${average_website2:.2f}")
    min_price, max_price = get_min_max(prices_website2)
    if min_price and max_price:
        print(f"Lowest price on {website2_name}: {min_price['name']} at ${min_price['price']:.2f}")
        print(f"Highest price on {website2_name}: {max_price['name']} at ${max_price['price']:.2f}")
    print(f"Average rating on {website2_name}: {average_rating_website2:.2f}")

    price_diff = abs(average_website1 - average_website2)
    print(f"\nPrice Comparison: {website1_name} is {'cheaper' if average_website1 < average_website2 else 'more expensive' if average_website1 > average_website2 else 'equally priced'} than {website2_name} by ${price_diff:.2f}")

    # Process website 1 prices
    min_price_website1, max_price_website1 = get_min_max(prices_website1)
    # Process website 2 prices
    min_price_website2, max_price_website2 = get_min_max(prices_website2)

    current_date = date.today().strftime('%m/%d/%Y')


    # After authorizing gspread
    sheet = client.open_by_url('URL_FOR_GOOGLE_SHEET')

    # Check if worksheet exists, create if not
    try:
        item_prices_ws = sheet.worksheet('item_prices') 
    except WorksheetNotFound:
        item_prices_ws = sheet.add_worksheet(title="item_prices", rows="1", cols="8")
            
    try:
        compared_prices_ws = sheet.worksheet('compared_prices') 
    except WorksheetNotFound:
        compared_prices_ws = sheet.add_worksheet(title="compared_prices", rows="1", cols="10")

    # Check if the sheets are empty before writing headers
    if len(item_prices_ws.get_all_values()) == 0:
        item_prices_ws.append_row(['ID', 'SearchID', 'Website', 'Item', 'Name', 'Price', 'Rating', 'Date Added'])
    if len(compared_prices_ws.get_all_values()) == 0:
        compared_prices_ws.append_row(['ID', 'SearchID', 'Website', 'Item', 'Lowest Price', 'Highest Price', 'Avg Price', 'Price Difference', 'Avg Rating', 'Date Added'])

    # First Google Sheet
    item_prices = [['SearchID', 'Website', 'Item', 'Name', 'Price', 'Rating', 'Date Added']]
    # Generate unique IDs for each item
    for i, item in enumerate(prices_website1, start=item_prices_max_id+1):
        item_prices.append([i, search_id, website1_name, word, item['name'], item['price'], item['rating'] if item['rating'] is not None else 0, current_date])
    for i, item in enumerate(prices_website2, start=len(prices_website1)+item_prices_max_id+1):
        item_prices.append([i, search_id, website2_name, word, item['name'], item['price'], item['rating'] if item['rating'] is not None else 0, current_date])


    # Second Google Sheet
    compared_prices = [['SearchID', 'Website', 'Item', 'Lowest Price', 'Highest Price', 'Avg Price', 'Price Difference', 'Avg Rating', 'Date Added']]
    if prices_website1:
        compared_prices.append([compared_prices_max_id+1, search_id, website1_name, word, min_price_website1['price'], max_price_website1['price'], average_website1, price_diff, average_rating_website1 if average_rating_website1 is not None else 0, current_date])
    if prices_website2:
        compared_prices.append([compared_prices_max_id+2, search_id, website2_name, word, min_price_website2['price'], max_price_website2['price'], average_website2, price_diff, average_rating_website2 if average_rating_website2 is not None else 0, current_date])


    # Append rows to item_prices
    for row in item_prices[1:]:  # Skip the header
        item_prices_ws.append_row(row)

    # Append rows to compared_prices
    for row in compared_prices[1:]:  # Skip the header
        compared_prices_ws.append_row(row)

    print(f"\nGoogle Sheet Updated")

search_id = get_search_id()
item_prices_max_id = get_max_id('item_prices')
compared_prices_max_id = get_max_id('compared_prices')
search_word = get_search_word_from_sheet()

if search_word is not None:
    compare_prices(search_word, item_prices_max_id, compared_prices_max_id)
else:
    print("No search word for today was found.")

In [ ]:
# Code for setting up local mysql server

from datetime import datetime
import gspread
import mysql.connector
from mysql.connector import Error
from oauth2client.service_account import ServiceAccountCredentials

# Define the scope
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']

# Add your Service Account File
creds = ServiceAccountCredentials.from_json_keyfile_name(r"PATH_TO_CREDENTIALS_FILE", scope)

# authorize your client
client = gspread.authorize(creds)

def create_database():
    conn = None
    try:
        conn = mysql.connector.connect(
            host="localhost",
            user="root",
            passwd="admin"
        )
        if conn.is_connected():
            cursor = conn.cursor()
            cursor.execute("CREATE DATABASE PricesDB")
            print('Database created successfully....')
    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()

create_database()


def create_connection():
    conn = None
    try:
        conn = mysql.connector.connect(
            host="localhost",
            user="root",
            passwd="admin",
            database="PricesDB"
        )
        if conn.is_connected():
            print('Connected to MySQL database')
    except Error as e:
        print(e)
    return conn


def create_table(conn):
    try:
        cursor = conn.cursor()
        item_prices_table_query = """
            CREATE TABLE IF NOT EXISTS item_prices (
                SEARCHID NUMBER(38,0),
                WEBSITE VARCHAR(255),
                ITEM VARCHAR(255),
                NAME VARCHAR(255),
                PRICE FLOAT,
                RATING FLOAT,
                DATEADDED DATE
            );
        """
        cursor.execute(item_prices_table_query)

        compared_prices_table_query = """
            CREATE TABLE IF NOT EXISTS compared_prices (
                SEARCHID NUMBER(38,0),
                WEBSITE VARCHAR(255),
                ITEM VARCHAR(255),
                LOWESTPRICE FLOAT,
                HIGHESTPRICE FLOAT,
                AVGPRICE FLOAT,
                PRICEDIFFERENCE FLOAT,
                AVGRATING FLOAT,
                DATEADDED DATE
            );
        """
        cursor.execute(compared_prices_table_query)

        print('Tables Updated...')

    except Error as e:
        print(e)


def search_id_exists(conn, search_id, table_name):
    query = f"SELECT 1 FROM {table_name} WHERE SearchID = {float(search_id)} LIMIT 1"
    try:
        cursor = conn.cursor()
        cursor.execute(query)
        return cursor.fetchone() is not None
    except Exception as e:
        print(e)
        return False
    
def get_max_search_id(conn, table_name):
    query = f"SELECT MAX(SearchID) FROM {table_name}"
    try:
        cursor = conn.cursor()
        cursor.execute(query)
        max_search_id = cursor.fetchone()[0]
        return max_search_id if max_search_id else 0
    except Exception as e:
        print(e)
        return 0


def insert_into_item_prices(conn, data):
    query = """
        INSERT INTO item_prices(SearchID, Website, Item, Name, Price, Rating, DateAdded)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
    """
    try:
        cursor = conn.cursor()
        
        # Get the maximum existing SearchID
        max_search_id = get_max_search_id(conn, "item_prices")
        
        # Filter out data with a SearchID higher than the maximum value
        filtered_data = [(row[0], row[1], row[2], row[3], row[4], row[5], row[6]) for row in data if int(row[0]) > max_search_id]
        
        if filtered_data:
            cursor.executemany(query, filtered_data)
            conn.commit()
            print(f"Inserted {len(filtered_data)} new rows into item_prices.")
        else:
            print("No new data to insert into item_prices.")

    except Exception as e:
        print(e)


def insert_into_compared_prices(conn, data):
    query = """
        INSERT INTO compared_prices(SearchID, Website, Item, LowestPrice, HighestPrice, AvgPrice, PriceDifference, AvgRating, DateAdded)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    try:
        cursor = conn.cursor()
        
        # Get the maximum existing SearchID
        max_search_id = get_max_search_id(conn, "compared_prices")
        
        # Filter out data with a SearchID higher than the maximum value
        filtered_data = [(row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8]) for row in data if int(row[0]) > max_search_id]
        
        if filtered_data:
            cursor.executemany(query, filtered_data)
            conn.commit()
            print(f"Inserted {len(filtered_data)} new rows into compared_prices.")
        else:
            print("No new data to insert into compared_prices.")

    except Exception as e:
        print(e)



def close_connection(conn):
    if conn:
        conn.close()
        print('Database connection closed.')


# Establish the connection
conn = create_connection()

# Create tables if they don't exist
create_table(conn)


def read_data_from_google_sheet(sheet_url):
    
    # Open the Google spreadsheet using its url
    sheet = client.open_by_url(sheet_url)
    
    # Select the first sheet in the spreadsheet
    worksheet = sheet.get_worksheet(0)
    
    # Get all data from the worksheet
    data = worksheet.get_all_values()
    return data

def read_gsheet_and_insert_item_prices(sheet_url, insert_function):
    data = read_data_from_google_sheet(sheet_url)
    items = []
    unique_search_ids = set()
    for row in data[1:]:  # Skip header row
        # Convert price related data to float and date to proper date format
        for i in [5]:  # Convert 'Price' to float
            row[i] = float(row[i])
        if '/' in row[7]:  # Convert 'Date Added' to date format
            row[7] = datetime.strptime(row[7], '%m/%d/%Y').date()
        items.append(row)
        unique_search_ids.add(row[1])

    items_to_insert = []  # Initialize the list here

    for search_id in unique_search_ids:
        if search_id_exists(conn, search_id, "item_prices"):
            print(f"SearchID {search_id} already exists. Skipping...")
            continue
        for item in items: # replace 'sorted_items' with 'items'
            if item[1] == search_id:
                items_to_insert.append(tuple(item[1:]))

    insert_function(conn, items_to_insert)

def read_gsheet_and_insert_compared_prices(sheet_url, insert_function):
    # Open the Google spreadsheet using its URL
    sheet = client.open_by_url(sheet_url)

    # Select the second sheet in the spreadsheet (index 1)
    worksheet = sheet.get_worksheet(1)

    # Get all data from the worksheet
    data = worksheet.get_all_values()

    # Find the index of the ID column in the header row
    header_row = data[0]
    id_column_index = header_row.index('ID')

    items = []
    unique_search_ids = set()
    for row in data[1:]:  # Skip header row
        # Convert price related data to float and date to proper date format
        if len(row) >= 8:  # Check if row has at least 8 columns
            for i in range(4, 8):  # Convert columns 4 to 7 to float
                row[i] = float(row[i])
            if '/' in row[9]:  # Convert 'Date Added' to date format
                row[9] = datetime.strptime(row[9], '%m/%d/%Y').date()
            items.append(row)
            unique_search_ids.add(row[1])

    items_to_insert = []

    for search_id in unique_search_ids:
        if search_id_exists(conn, search_id, "compared_prices"):
            print(f"SearchID {search_id} already exists. Skipping...")
            continue
        for item in items: # replace 'sorted_items' with 'items'
            if item[1] == search_id:
                items_to_insert.append(tuple(item[1:]))
                
    insert_function(conn, items_to_insert)


# Establish the Snowflake connection
conn = create_connection()

# Create tables if they don't exist
create_table(conn)

# Read and insert data for each sheet
read_gsheet_and_insert_item_prices('URL_FOR_GOOGLE_SHEET', insert_into_item_prices)
read_gsheet_and_insert_compared_prices('URL_FOR_GOOGLE_SHEET', insert_into_compared_prices)

print(f"Data Upload Completed")


In [ ]:
# Code for setting up and sending data to Snowflake DB

# Adding data to our Snowflake Database
def create_connection():
    conn = None
    try:
        conn = snowflake.connector.connect(
            user="USERNAME",
            password="PASSWORD",
            account="SNOWFLAKE IDENTIFIER"
        )
        print('Connected to Snowflake database')
    except Exception as e:
        print(e)
    return conn


def create_table(conn):
    try:
        cursor = conn.cursor()
        cursor.execute("USE DATABASE PRICE_DATABASE")
        item_prices_table_query = """
            CREATE TABLE IF NOT EXISTS PRICE_DATABASE.PUBLIC.ITEM_PRICES (
                SEARCHID NUMBER(38,0),
                WEBSITE VARCHAR(16777216),
                ITEM VARCHAR(16777216),
                NAME VARCHAR(16777216),
                PRICE FLOAT,
                RATING FLOAT,
                DATEADDED DATE
            );
        """
        cursor.execute(item_prices_table_query)

        compared_prices_table_query = """
            CREATE TABLE IF NOT EXISTS PRICE_DATABASE.PUBLIC.COMPARED_PRICES (
                SEARCHID NUMBER(38,0),
                WEBSITE VARCHAR(16777216),
                ITEM VARCHAR(16777216),
                LOWESTPRICE FLOAT,
                HIGHESTPRICE FLOAT,
                AVGPRICE FLOAT,
                PRICEDIFFERENCE FLOAT,
                AVGRATING FLOAT,
                DATEADDED DATE
            );
        """
        cursor.execute(compared_prices_table_query)

        print('Tables Updated...')

    except Exception as e:
        print(e)


def search_id_exists(conn, search_id, table_name):
    query = f"SELECT 1 FROM {table_name} WHERE SearchID = {float(search_id)} LIMIT 1"
    try:
        cursor = conn.cursor()
        cursor.execute(query)
        return cursor.fetchone() is not None
    except Exception as e:
        print(e)
        return False
    
def get_max_search_id(conn, table_name):
    query = f"SELECT MAX(SearchID) FROM {table_name}"
    try:
        cursor = conn.cursor()
        cursor.execute(query)
        max_search_id = cursor.fetchone()[0]
        return max_search_id if max_search_id else 0
    except Exception as e:
        print(e)
        return 0


def insert_into_item_prices(conn, data):
    query = """
        INSERT INTO item_prices(SearchID, Website, Item, Name, Price, Rating, DateAdded)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
    """
    try:
        cursor = conn.cursor()
        
        # Get the maximum existing SearchID
        max_search_id = get_max_search_id(conn, "item_prices")
        
        # Filter out data with a SearchID higher than the maximum value
        filtered_data = [(row[0], row[1], row[2], row[3], row[4], row[5], row[6]) for row in data if int(row[0]) > max_search_id]
        
        if filtered_data:
            cursor.executemany(query, filtered_data)
            conn.commit()
            print(f"Inserted {len(filtered_data)} new rows into item_prices.")
        else:
            print("No new data to insert into item_prices.")

    except Exception as e:
        print(e)


def insert_into_compared_prices(conn, data):
    query = """
        INSERT INTO compared_prices(SearchID, Website, Item, LowestPrice, HighestPrice, AvgPrice, PriceDifference, AvgRating, DateAdded)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    try:
        cursor = conn.cursor()
        
        # Get the maximum existing SearchID
        max_search_id = get_max_search_id(conn, "compared_prices")
        
        # Filter out data with a SearchID higher than the maximum value
        filtered_data = [(row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8]) for row in data if int(row[0]) > max_search_id]
        
        if filtered_data:
            cursor.executemany(query, filtered_data)
            conn.commit()
            print(f"Inserted {len(filtered_data)} new rows into compared_prices.")
        else:
            print("No new data to insert into compared_prices.")

    except Exception as e:
        print(e)


def close_connection(conn):
    if conn:
        conn.close()
        print('Database connection closed.')


# Establish the Snowflake connection
conn = create_connection()

# Create tables if they don't exist
create_table(conn)


def read_data_from_google_sheet(sheet_url):
    
    # Open the Google spreadsheet using its url
    sheet = client.open_by_url(sheet_url)
    
    # Select the first sheet in the spreadsheet
    worksheet = sheet.get_worksheet(0)
    
    # Get all data from the worksheet
    data = worksheet.get_all_values()
    return data

def read_gsheet_and_insert_item_prices(sheet_url, insert_function):
    data = read_data_from_google_sheet(sheet_url)
    items = []
    unique_search_ids = set()
    for row in data[1:]:  # Skip header row
        # Convert price related data to float and date to proper date format
        for i in [5]:  # Convert 'Price' to float
            row[i] = float(row[i])
        if '/' in row[7]:  # Convert 'Date Added' to date format
            row[7] = datetime.strptime(row[7], '%m/%d/%Y').date()
        items.append(row)
        unique_search_ids.add(row[1])

    items_to_insert = []  # Initialize the list here

    for search_id in unique_search_ids:
        if search_id_exists(conn, search_id, "item_prices"):
            print(f"SearchID {search_id} already exists. Skipping...")
            continue
        for item in items: # replace 'sorted_items' with 'items'
            if item[1] == search_id:
                items_to_insert.append(tuple(item[1:]))

    insert_function(conn, items_to_insert)

def read_gsheet_and_insert_compared_prices(sheet_url, insert_function):
    # Open the Google spreadsheet using its URL
    sheet = client.open_by_url(sheet_url)

    # Select the second sheet in the spreadsheet (index 1)
    worksheet = sheet.get_worksheet(1)

    # Get all data from the worksheet
    data = worksheet.get_all_values()

    # Find the index of the ID column in the header row
    header_row = data[0]
    id_column_index = header_row.index('ID')

    items = []
    unique_search_ids = set()
    for row in data[1:]:  # Skip header row
        # Convert price related data to float and date to proper date format
        if len(row) >= 8:  # Check if row has at least 8 columns
            for i in range(4, 8):  # Convert columns 4 to 7 to float
                row[i] = float(row[i])
            if '/' in row[9]:  # Convert 'Date Added' to date format
                row[9] = datetime.strptime(row[9], '%m/%d/%Y').date()
            items.append(row)
            unique_search_ids.add(row[1])

    items_to_insert = []

    for search_id in unique_search_ids:
        if search_id_exists(conn, search_id, "compared_prices"):
            print(f"SearchID {search_id} already exists. Skipping...")
            continue
        for item in items: # replace 'sorted_items' with 'items'
            if item[1] == search_id:
                items_to_insert.append(tuple(item[1:]))
                
    insert_function(conn, items_to_insert)


# Establish the Snowflake connection
conn = create_connection()

# Create tables if they don't exist
create_table(conn)

# Read and insert data for each sheet
read_gsheet_and_insert_item_prices('URL_FOR_GOOGLE_SHEET', insert_into_item_prices)
read_gsheet_and_insert_compared_prices('URL_FOR_GOOGLE_SHEET', insert_into_compared_prices)

print(f"Data Upload Completed")